# MySQL UNION 操作符用于连接两个以上的 SELECT 语句的结果组合到一个结果集合中。多个 SELECT 语句会删除重复的数据。

请注意，UNION 内部的 SELECT 语句必须拥有相同数量的列。列也必须拥有相似的数据类型。同时，每条 SELECT 语句中的列的顺序必须相同
http://www.w3school.com.cn/sql/sql_union.asp

In [9]:
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import text 
engine=create_engine('mysql+pymysql://ACCOUNT:PASSWORE@IP:3306/dw')
sql = "show tables" 
df = pd.read_sql_query(sql, engine)
df.head()

,Tables_in_dw
0,dim_category
1,dim_date
2,dim_goods
3,dim_member
4,dim_shop


# 或者

In [10]:
import pymysql
# 打开数据库连接
db = pymysql.connect("118.190.201.131","account","PASSWORD","dw" ) #,"
# 使用 cursor() 方法创建一个游标对象 cursor
cursor = db.cursor()
# 使用 execute() 方法执行 SQL，如果表存在则删除
sql_2="SHOW TABLES"
cursor.execute(sql_2)
result2=cursor.fetchall()
print('result2:',result2) 
# 关闭数据库连接
db.close()

result2: (('dim_category',), ('dim_date',), ('dim_goods',), ('dim_member',), ('dim_shop',), ('fct_sales',), ('fct_sales_item',))


/*
* 多表操作之：表的行合并
* 用 union
* 1、合并的表的列数必须是一致，也就是合并的列数相同
* 2、union 与 union all 对二个查询结果合并的时候，如果有结果有重复，union 会把重复
的值 删除 ；
* 3、union 与 union all 

In [12]:
sql=text('''
-- 统计 20170703-20170710 周内每天及本周累计销售金额、订单量、会员数、订单占比
select date_format(dimDateID,'%W') as week_day
,sum(AMT) as sales_money
,count(distinct salesID) as order_number
,count(distinct dimMemberID) as member_number
,count(distinct salesID)/(select count(distinct salesID) from dw.fct_sales
where dimDateID between '20170703' and '20170710')
as order_rate
from dw.fct_sales
where dimDateID between '20170703' and '20170710'
group by date_format(dimDateID,'%W')
''')
df = pd.read_sql_query(sql, engine)
df.head()

,week_day,sales_money,order_number,member_number,order_rate
0,Friday,96602.58,1739,827,0.1172
1,Monday,236153.77,4231,1637,0.2851
2,Saturday,145862.13,1988,871,0.1340
3,Sunday,128228.18,2016,926,0.1358
4,Thursday,116434.51,1805,802,0.1216


In [17]:
sql=text('''
select sum(AMT) as sales_money
,count(distinct salesID) as order_number
,count(distinct dimMemberID) as member_number
from dw.fct_sales
where dimDateID between '20170703' and '20170710';
''')
df = pd.read_sql_query(sql, engine)
df

,sales_money,order_number,member_number
0,888178.6,14840,4188


In [16]:
#---- 用 union 合并在一起
sql=text('''
select date_format(dimDateID,'%W') as week_day
,sum(AMT) as sales_money
,count(distinct salesID) as order_number
,count(distinct dimMemberID) as member_number
,count(distinct salesID)/(select count(distinct salesID) from dw.fct_sales
where dimDateID between '20170703' and '20170710')
as order_rate
from dw.fct_sales
where dimDateID between '20170703' and '20170709'
group by date_format(dimDateID,'%W')
union
select week(dimDateID,1) as week_1
,sum(AMT) as sales_money
,count(distinct salesID) as order_number
,count(distinct dimMemberID) as member_number
,'100%' as total
from dw.fct_sales
where dimDateID between '20170703' and '20170709'
group by week(dimDateID,1);
''')
df = pd.read_sql_query(sql, engine)
df

,week_day,sales_money,order_number,member_number,order_rate
0,Friday,96602.58,1739,827,0.117183288
1,Monday,123386.19,2183,939,0.147102425
2,Saturday,145862.13,1988,871,0.133962264
3,Sunday,128228.18,2016,926,0.135849056
4,Thursday,116434.51,1805,802,0.121630727
5,Tuesday,63387.56,1201,612,0.080929919
6,Wednesday,101509.87,1860,886,0.125336927
7,27,775411.02,12792,3828,100%


In [18]:
#---- 用 union all 合并在一起
sql=text('''
select date_format(dimDateID,'%W') as week_day
,sum(AMT) as sales_money
,count(distinct salesID) as order_number
,count(distinct dimMemberID) as member_number
,count(distinct salesID)/(select count(distinct salesID) from dw.fct_sales
where dimDateID between '20170703' and '20170710')
as order_rate
from dw.fct_sales
where dimDateID between '20170703' and '20170709'
group by date_format(dimDateID,'%W')
union all
select week(dimDateID,1) as week_1
,sum(AMT) as sales_money
,count(distinct salesID) as order_number
,count(distinct dimMemberID) as member_number
,'100%' as total
from dw.fct_sales
where dimDateID between '20170703' and '20170709'
group by week(dimDateID,1);
''')
df = pd.read_sql_query(sql, engine)
df

,week_day,sales_money,order_number,member_number,order_rate
0,Friday,96602.58,1739,827,0.117183288
1,Monday,123386.19,2183,939,0.147102425
2,Saturday,145862.13,1988,871,0.133962264
3,Sunday,128228.18,2016,926,0.135849056
4,Thursday,116434.51,1805,802,0.121630727
5,Tuesday,63387.56,1201,612,0.080929919
6,Wednesday,101509.87,1860,886,0.125336927
7,27,775411.02,12792,3828,100%
